# SageMakerのSerializerとDeserializerを理解する

このノートブックでは、推論エンドポイントにNumpyArrayとCSV形式でのリクエストを想定して、
NumpyArrayデータとCSVデータがどのようにシリアライズされるか、どのようにデシリアライズされるかを学びます。

SageMakerのシリアライズ/デシリアライズの仕組みを知ることで、より推論の仕組みを深く理解します。

ノートブックは1分程度で実行できます。

## SageMakerの動作

詳しくは、BlackBelt動画をご覧ください：https://www.youtube.com/watch?v=sngNd79GpmE

* クライアント側でデータがシリアライズされる。シリアルデータを推論エンドポイントに送る。
* ====== SageMaker 内部 ==========
* 推論エンドポイントは、シリアル化されたデータを受け取る
* SageMakerのコードで、デシリアライズする。
* input_fn実行
* predict_fn実行
* output_fn実行
* データをシリアライズする。
* クライアントに送信
* ====== SageMaker 内部 ==========
* クライアント側で、デシリアライズする。

# 1. Serializerの理解

Serializerは、推論エンドポイントに対して、シリアライズしたデータと、CONTENT_TYPEを提供します。

Serializerのコードは、GitHubで確認することができます。

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/serializers.py

ドキュメント：https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html


input形式は様々

output形式はSerializerで指定したクラスに依存

## 1-1. データ準備

Serializerを通すデータとして、以下のパターンを試します。

* ndarray型
* string型

In [1]:
import numpy as np

In [2]:
# ndarray型の推論エンドポイントへのインプットデータ
data_ndarr = np.array([[0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81],
                       [0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05],
                       [4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66]])

# string型の推論エンドポイントへのインプットデータ
data_str = '0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'


In [3]:
# 確認
print(type(data_ndarr))
print('=' * 50)
print(data_ndarr)
print('=' * 50)
data_ndarr

<class 'numpy.ndarray'>
[[2.53870e-01 0.00000e+00 6.91000e+00 0.00000e+00 4.48000e-01 5.39900e+00
  9.53000e+01 5.87000e+00 3.00000e+00 2.33000e+02 1.79000e+01 3.96900e+02
  3.08100e+01]
 [1.95100e-02 1.75000e+01 1.38000e+00 0.00000e+00 4.16100e-01 7.10400e+00
  5.95000e+01 9.22290e+00 3.00000e+00 2.16000e+02 1.86000e+01 3.93240e+02
  8.05000e+00]
 [4.64689e+00 0.00000e+00 1.81000e+01 0.00000e+00 6.14000e-01 6.98000e+00
  6.76000e+01 2.53290e+00 2.40000e+01 6.66000e+02 2.02000e+01 3.74680e+02
  1.16600e+01]]


array([[2.53870e-01, 0.00000e+00, 6.91000e+00, 0.00000e+00, 4.48000e-01,
        5.39900e+00, 9.53000e+01, 5.87000e+00, 3.00000e+00, 2.33000e+02,
        1.79000e+01, 3.96900e+02, 3.08100e+01],
       [1.95100e-02, 1.75000e+01, 1.38000e+00, 0.00000e+00, 4.16100e-01,
        7.10400e+00, 5.95000e+01, 9.22290e+00, 3.00000e+00, 2.16000e+02,
        1.86000e+01, 3.93240e+02, 8.05000e+00],
       [4.64689e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.14000e-01,
        6.98000e+00, 6.76000e+01, 2.53290e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.74680e+02, 1.16600e+01]])

In [4]:
# 確認
print(type(data_str))
print('=' * 50)
print(data_str)
print('=' * 50)
data_str

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

In [5]:
# 参考：推論実行で使っていたコード
#with open(local_test, 'r') as f:
#    payload = f.read().strip()
#    print(type(payload))
#    print(payload)
#print('=' * 20)
#payload

## 1-2. ndarray型のデータをSerializerに適用する

In [6]:
from sagemaker.serializers import CSVSerializer
from sagemaker.serializers import NumpySerializer
from sagemaker.serializers import JSONSerializer

In [7]:
# Serializerのデフォルト CONTENT_TYPE を確認
print(CSVSerializer().content_type)
print(NumpySerializer().content_type)
print(JSONSerializer().content_type)

text/csv
application/x-npy
application/json


### 1-2-1. CSVSerializerでシリアライズする

In [8]:
serialized = CSVSerializer().serialize(data_ndarr)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

### 1-2-2. NumpySerializerでシリアライズする

In [9]:
serialized = NumpySerializer().serialize(data_ndarr)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'bytes'>
b'\x93NUMPY\x01\x00v\x00{\'descr\': \'<f8\', \'fortran_order\': False, \'shape\': (3, 13), }                                                         \n\x8f\xdf\xdb\xf4g?\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\xa4p=\n\xd7\xa3\x1b@\x00\x00\x00\x00\x00\x00\x00\x00y\xe9&1\x08\xac\xdc?\x7fj\xbct\x93\x98\x15@33333\xd3W@{\x14\xaeG\xe1z\x17@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00 m@fffff\xe61@fffff\xcex@\x8f\xc2\xf5(\\\xcf>@\x98\xa3\xc7\xefm\xfa\x93?\x00\x00\x00\x00\x00\x801@\x14\xaeG\xe1z\x14\xf6?\x00\x00\x00\x00\x00\x00\x00\x00\xfee\xf7\xe4a\xa1\xda?\xd1"\xdb\xf9~j\x1c@\x00\x00\x00\x00\x00\xc0M@\x8a\x8e\xe4\xf2\x1fr"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00k@\x9a\x99\x99\x99\x99\x992@\xa4p=\n\xd7\x93x@\x9a\x99\x99\x99\x99\x19 @\x11p\x08Uj\x96\x12@\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x99\x99\x99\x99\x192@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xa5\xe3?\xecQ\xb8\x1e\x85\xeb\x1b@fffff\xe6P@\xa5N@\x13aC\x04@\x00\x00\x00\x00\x00\x008@\x00\x00\x00\x00

b'\x93NUMPY\x01\x00v\x00{\'descr\': \'<f8\', \'fortran_order\': False, \'shape\': (3, 13), }                                                         \n\x8f\xdf\xdb\xf4g?\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\xa4p=\n\xd7\xa3\x1b@\x00\x00\x00\x00\x00\x00\x00\x00y\xe9&1\x08\xac\xdc?\x7fj\xbct\x93\x98\x15@33333\xd3W@{\x14\xaeG\xe1z\x17@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00 m@fffff\xe61@fffff\xcex@\x8f\xc2\xf5(\\\xcf>@\x98\xa3\xc7\xefm\xfa\x93?\x00\x00\x00\x00\x00\x801@\x14\xaeG\xe1z\x14\xf6?\x00\x00\x00\x00\x00\x00\x00\x00\xfee\xf7\xe4a\xa1\xda?\xd1"\xdb\xf9~j\x1c@\x00\x00\x00\x00\x00\xc0M@\x8a\x8e\xe4\xf2\x1fr"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00k@\x9a\x99\x99\x99\x99\x992@\xa4p=\n\xd7\x93x@\x9a\x99\x99\x99\x99\x19 @\x11p\x08Uj\x96\x12@\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x99\x99\x99\x99\x192@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xa5\xe3?\xecQ\xb8\x1e\x85\xeb\x1b@fffff\xe6P@\xa5N@\x13aC\x04@\x00\x00\x00\x00\x00\x008@\x00\x00\x00\x00\x00\xd0\x84@333

## 1-2-3. JSONSerializerでシリアライズする

In [10]:
serialized = JSONSerializer().serialize(data_ndarr)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
[[0.25387, 0.0, 6.91, 0.0, 0.448, 5.399, 95.3, 5.87, 3.0, 233.0, 17.9, 396.9, 30.81], [0.01951, 17.5, 1.38, 0.0, 0.4161, 7.104, 59.5, 9.2229, 3.0, 216.0, 18.6, 393.24, 8.05], [4.64689, 0.0, 18.1, 0.0, 0.614, 6.98, 67.6, 2.5329, 24.0, 666.0, 20.2, 374.68, 11.66]]


'[[0.25387, 0.0, 6.91, 0.0, 0.448, 5.399, 95.3, 5.87, 3.0, 233.0, 17.9, 396.9, 30.81], [0.01951, 17.5, 1.38, 0.0, 0.4161, 7.104, 59.5, 9.2229, 3.0, 216.0, 18.6, 393.24, 8.05], [4.64689, 0.0, 18.1, 0.0, 0.614, 6.98, 67.6, 2.5329, 24.0, 666.0, 20.2, 374.68, 11.66]]'

## 1.3 string型のデータをシリアライザに適用する

### 1-3-1. CSVSerializerでシリアライズする

In [11]:
serialized = CSVSerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

### 1-3-2. NumpySerializerでシリアライズする

In [12]:
serialized = NumpySerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'bytes'>
b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00

b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x000\x00\x00\x0

## 1-3-3. JSONSerializerでシリアライズする

In [13]:
serialized = JSONSerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
"0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66"


'"0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66"'

# 2. Deserializerの理解
Deserializerは、推論エンドポイントからの応答（シリアルデータ）をデシリアライズすることと、ACCEPTが設定されており、推論エンドポイントからどの形式でデータを受領するかを定義できます。これは、output_fn()での出力と合わせる必要があります。


GitHubのソースコード

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/deserializers.py

ドキュメント：https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html

inputは、推論エンドポイントから受信したシリアルデータ。形式はJSONやndarray

outputは、クラスで指定した形式（JSON, ndarray, pandasなど）


LightGBMは推論結果をndarray型で出力するので、ndarray型をシリアライズして、クライアントに渡すことを想定する。

## 2-1. データ準備

Deserializerを通すデータとして、以下のパターンを試します。

* ndarray型
* json型

In [14]:
# 返却したいオブジェクト:ndarray型
body_ndarr = np.array([
                       19.95642073217597,
                       27.844891841022335,
                       23.747437427003455
                      ])

# 返却したいオブジェクト:JSON
body_json = {
    "1": 19.95642073217597,
    "2": 27.844891841022335,
    "3": 23.747437427003455
}

In [15]:
# 確認
body_ndarr

array([19.95642073, 27.84489184, 23.74743743])

In [16]:
# 確認
body_json

{'1': 19.95642073217597, '2': 27.844891841022335, '3': 23.747437427003455}

## 2-2. ndarray型のデータをDeserializerに適用する

ndarray型のデータは、NumpyDeserializerでDeserializeすることができます

https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.NumpyDeserializer

Deserialize a stream of data in .npy or UTF-8 CSV/JSON format to a numpy array.

In [17]:
from sagemaker.deserializers import PandasDeserializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import NumpyDeserializer

In [18]:
# Deserializerのデフォルト ACCEPT を確認
print(PandasDeserializer().ACCEPT)
print(JSONDeserializer().ACCEPT)
print(NumpyDeserializer().ACCEPT)

('text/csv', 'application/json')
('application/json',)
('application/x-npy',)


In [19]:
import botocore
import json
from io import BytesIO

### 2-2-1. NumpyDeserializerでDeserializeする

In [20]:
# シリアライズデータを作成
body_ndarr_stream = botocore.response.StreamingBody(BytesIO(body_ndarr.dumps()),len(body_ndarr.dumps()))

# デシリアライズ
deserialized = NumpyDeserializer().deserialize(body_ndarr_stream, 'application/x-npy')

# 確認
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'numpy.ndarray'>
[19.95642073 27.84489184 23.74743743]


array([19.95642073, 27.84489184, 23.74743743])

## 2-3. JSON型のデータをDeserializerに適用する

### 2-3-1. PandasDeserializerでDeserializeする

In [21]:
# エンコード。(encode()はデフォルトでutf-8。)
body_json_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body_json_stream = botocore.response.StreamingBody(BytesIO(body_json_encoded),len(body_json_encoded))

# デシリアライズ
deserialized = PandasDeserializer().deserialize(body_json_stream, 'text/csv')

# 確認
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: [{"1": 19.95642073217597,  "2": 27.844891841022335,  "3": 23.747437427003455}]
Index: []


,"{""1"": 19.95642073217597","""2"": 27.844891841022335","""3"": 23.747437427003455}"


### 2-3-2. JSONDeserializerでDeserializeする

In [22]:
# エンコード(encode()はデフォルトでutf-8)
body_json_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body_json_stream = botocore.response.StreamingBody(BytesIO(body_json_encoded),len(body_json_encoded))

# デシリアライズ
deserialized = JSONDeserializer().deserialize(body_json_stream, 'text/csv')

# 確認
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'dict'>
{'1': 19.95642073217597, '2': 27.844891841022335, '3': 23.747437427003455}


{'1': 19.95642073217597, '2': 27.844891841022335, '3': 23.747437427003455}

### 2-3-3. NumpyDeserializerでDeserializeする

In [23]:
# エンコード(encode()はデフォルトでutf-8)
body_json_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body_json_stream = botocore.response.StreamingBody(BytesIO(body_json_encoded),len(body_json_encoded))

# デシリアライズ
deserialized = NumpyDeserializer(dtype='str').deserialize(body_json_stream, 'text/csv')

# 確認
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'numpy.ndarray'>
['{"1": 19.95642073217597' ' "2": 27.844891841022335'
 ' "3": 23.747437427003455}']


array(['{"1": 19.95642073217597', ' "2": 27.844891841022335',
       ' "3": 23.747437427003455}'], dtype='<U25')

# END OF CONTAINTS ==========

# 参考

botocore.response

https://botocore.amazonaws.com/v1/documentation/api/latest/reference/response.html

raw_streamを入力する必要がある。


バイナリ I/O
https://docs.python.org/ja/3/library/io.html#binary-i-o


BytesIO はインメモリーのバイナリストリームです:

f = io.BytesIO(b"some initial binary data: \x00\x01")

BytesIOはseekもreadもできる。